In [ ]:
import torch
%run data_dif.ipynb
%run trainer_dif.ipynb
%run utils.ipynb
import pickle
from pathlib import Path

In [ ]:
class DetectGenerativeModel:
    self.fingerprints = None
    self.generative_models = None

In [ ]:
class GenerativeModel(TrainerMultiple):
    def __init__(self, fingerprint_dir, image_dir, epoch=0, batch=64):
        model_ep = epoch
        images_dir = Path(image_dir)
        check_dir = Path(fingerprint_dir)
        check_existence(check_dir, False)
        check_existence(images_dir, False)
        with open(check_dir / "train_hypers.pt", 'rb') as pickle_file:
            hyperparams = pickle.load(pickle_file)
        hyperparams['Device'] = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        hyperparams['Batch Size'] = batch
        super().__init__(hyperparams)

    def get_correlation(self, img_path):
        image = load_pil_image(img_path, self.crop_size)
        image = np.array(image)
        image = torch.tensor(image.transpose((2, 0, 1))).type(torch.float32).div(255)
        residuals = self.denoiser.denoise([image]).detach()
        alpha = (1 - self.alpha) * torch.rand((len(images), 1, 1, 1)).to(self.device) + self.alpha
        residuals = alpha * residuals
        residuals = torch.cat((residuals, f_mean, r_mean), dim=0)
        corr = self.corr_fun(self.fingerprint, residuals)
        return corr[0]

a = GenerativeModel(fingerprint_dir=0, batch=50)
isintance(a, image)
# def __instancecheck__(self, img_path):


In [ ]:
def load_pil_image(img_path, img_size=None):
    img = Image.open(img_path).convert('RGB')

    if img_size is not None:
        w, h = img.size
        left = (w - img_size[1]) / 2
        top = (h - img_size[0]) / 2
        right = (w + img_size[1]) / 2
        bottom = (h + img_size[0]) / 2

        img = img.crop((left, top, right, bottom))

    return img